In [15]:
import numpy as np
import cv2 as cv
from scipy import linalg as la
from sklearn.utils import check_array, as_float_array
from skimage.measure import compare_psnr

class CCIPCA(object):
        
    def __init__(self, n_components=2, amnesic=2.0, copy=True):
        self.n_components = n_components
        if self.n_components < 2:
            raise ValueError ("must specifiy n_components for CCIPCA")
            
        self.copy = copy
        self.amnesic = amnesic
        self.iteration = 0

    def fit(self, X):        
        X = check_array(X)
        n_samples, n_features = X.shape 
        X = as_float_array(X, copy=self.copy)
        
        # init
        if self.iteration == 0:  
            self.mean_ = np.zeros([n_features], np.float)                           #行向量与行向量相加
            self.components_ = np.zeros([self.n_components,n_features], np.float)
        else:
            if n_features != self.components_.shape[1]:
                raise ValueError('The dimensionality of the new data and the existing components_ does not match')   
        
        # incrementally fit the model
        for i in range(0,n_samples):
            self.partial_fit(X[i,:])
        
        # update explained_variance_ratio_
        self.explained_variance_ratio_ = np.sqrt(np.sum(self.components_**2,axis=1))
        
        # sort by explained_variance_ratio_
        idx = np.argsort(-self.explained_variance_ratio_)
        self.explained_variance_ratio_ = self.explained_variance_ratio_[idx]
        self.components_ = self.components_[idx,:]
        
        # re-normalize
        self.explained_variance_ratio_ = (self.explained_variance_ratio_ / self.explained_variance_ratio_.sum())
           
        for r in range(0,self.components_.shape[0]):  #归一化
            self.components_[r,:] /= np.sqrt(np.dot(self.components_[r,:],self.components_[r,:]))
        #self.components_t = np.zeros((40,self.components_.shape[1]))
        return self
        
    def _amnestic(self, t):               # amnestic function
        if t <= int(self.amnesic):
            _rr = float(t+2-1)/float(t+2)    
            _lr = float(1)/float(t+2)    
        else:
            _rr = float(t+2-self.amnesic)/float(t+2)    
            _lr = float(1+self.amnesic)/float(t+2)
        
        return [_rr, _lr]
    

    def partial_fit(self, u):
        n = float(self.iteration)
        V = self.components_
        w1,w2=self._amnestic(n)
        self.mean_ = float(n+1-1)/float(n+1)*self.mean_ + float(1)/float(n+1)*u
        if n != 0:
        # mean center u        
            u = u - self.mean_
    
        # update components

        for j in range(0,self.n_components):
            
            if j > n:
                # the component has already been init to a zerovec
                pass
            
            elif j == n:
                # set the component to u 
                V[j,:] = u
                normedV = V[j,:] / la.norm(V[j,:])
            else:       
                # update the components
                V[j,:] = w1*V[j,:] + w2*np.dot(u,V[j,:])*u / la.norm(V[j,:])
                
                normedV = V[j,:] / la.norm(V[j,:])
            
            u = u - np.dot(np.dot(u.T,normedV),normedV)

        self.iteration += 1
        self.components_ = V
        return
    
    def transform(self, X):
        X = check_array(X)
        X_transformed = X - self.mean_
        X_transformed = np.dot(X_transformed, self.components_.T)
        return X_transformed

    def predict(self, X):
        decompass_data = np.dot(X, self.components_) + self.mean_
        return decompass_data


In [16]:
from PIL import Image
import numpy as np

In [17]:
image=Image.open('1.bmp')
image=np.array(image)
image.shape

(112, 92)

In [18]:
a=CCIPCA(3)
a.fit(image)
compass_data=a.transform(image)
decompass_data=a.predict(compass_data)
decompass_data.shape

(112, 92)

In [19]:
from skimage.measure import compare_psnr

In [20]:
decompass_data

array([[103.75915778, 102.68844866, 100.84189246, ...,  80.64248916,
         82.16808103,  82.18658795],
       [105.96837617, 105.0024789 , 103.13975457, ...,  82.54237202,
         84.17898215,  84.23389011],
       [107.00808313, 106.14371518, 104.27492342, ...,  83.47364324,
         85.26773058,  85.34115072],
       ...,
       [ 78.28566865,  76.46873736,  71.89449808, ...,  58.69746733,
         63.38279917,  64.48195152],
       [ 77.93751562,  76.31575438,  71.90204123, ...,  58.56772231,
         63.4804853 ,  64.50238115],
       [ 77.52760734,  75.82247989,  71.35919259, ...,  58.16336962,
         62.98741639,  64.02663034]])

In [21]:
#变成矩阵
ccimage=np.uint8(np.mat(decompass_data))
ccimage

matrix([[103, 102, 100, ...,  80,  82,  82],
        [105, 105, 103, ...,  82,  84,  84],
        [107, 106, 104, ...,  83,  85,  85],
        ...,
        [ 78,  76,  71, ...,  58,  63,  64],
        [ 77,  76,  71, ...,  58,  63,  64],
        [ 77,  75,  71, ...,  58,  62,  64]], dtype=uint8)

In [8]:
import cv2 as cv
cv.imshow('ccipca',ccimage)
cv.namedWindow('ccipca', cv.WINDOW_NORMAL)
cv.resizeWindow('ccipca',300, 300)
cv.waitKey(0)
cv.destoryAllWindows()

AttributeError: module 'cv2.cv2' has no attribute 'destoryAllWindows'

In [22]:
import cv2 as cv
cv.imshow('ccipca',ccimage)
cv.namedWindow('ccipca', cv.WINDOW_NORMAL)
cv.resizeWindow('ccipca',300, 300)
cv.waitKey(0)
cv.destoryAllWindows()

AttributeError: module 'cv2.cv2' has no attribute 'destoryAllWindows'

In [23]:
print("信噪比：",compare_psnr(image,ccimage))

信噪比： 19.97769245945037
